In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


data_dir = 'images'  
img_height, img_width = 128, 128  # Розмір зображення
batch_size = 16


datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    validation_split = 0.2  # 20% для валідації
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=42
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 класи
])

# Компіляція моделі
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=20, 
    validation_data=validation_generator,
    verbose=1
)


model.save('cnn_from_scratch.h5')

# Оцінка на тестових даних
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


Found 96 images belonging to 3 classes.


Found 24 images belonging to 3 classes.
Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.3963 - loss: 1.3341 - val_accuracy: 0.3333 - val_loss: 1.0251
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5756 - loss: 0.9875 - val_accuracy: 0.6667 - val_loss: 0.7355
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.6796 - loss: 0.7177 - val_accuracy: 0.6667 - val_loss: 0.6078
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7763 - loss: 0.5908 - val_accuracy: 0.8750 - val_loss: 0.3932
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8536 - loss: 0.4062 - val_accuracy: 0.8750 - val_loss: 0.3852
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8579 - loss: 0.3111 - val_accuracy: 0.8750 - val_loss: 0.3700
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9198 - loss: 0.3309 - val_accuracy: 0.8750 - val_loss: 0.3207
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9101 - loss: 0.2795 - val_accuracy: 0

Found 120 images belonging to 3 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 798ms/step - accuracy: 0.9400 - loss: 0.1388
Test Accuracy: 95.00%


transfer learning

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


data_dir = 'images'
img_height, img_width = 128, 128  # Розмір зображення
batch_size = 16

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Попередня обробка для MobileNetV2
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=42
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42
)

# MobileNetV2
base_model = MobileNetV2(
    input_shape=(img_height, img_width, 3),
    include_top=False,  # Не використовуємо верхній класифікаційний шар
    weights='imagenet'  # Попередньо навчені ваги
)


base_model.trainable = False


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Усереднення характеристик
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 класи
])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=10,  # Менше епох, оскільки модель вже навчена
    validation_data=validation_generator,
    verbose=1
)


model.save('cnn_transfer_learning.h5')

#Оцінка моделі
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy with Transfer Learning: {test_acc * 100:.2f}%")


Found 96 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.6518 - loss: 0.8915 - val_accuracy: 0.9583 - val_loss: 0.1622
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9504 - loss: 0.1493 - val_accuracy: 1.0000 - val_loss: 0.0363
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9682 - loss: 0.0626 - val_accuracy: 1.0000 - val_loss: 0.0112
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9597 - loss: 0.0606 - val_accuracy: 1.0000 - val_loss: 0.0254
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9871 - loss: 0.0468 - val_accuracy: 1.0000 - val_loss: 0.0357
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0150 - val_accuracy: 1.0000 - val_loss: 0.0173
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9952 - loss: 0.0090 - val_accuracy: 1.0000 - val_loss: 0.0116
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 1.0000 - val_loss: 0.0120
Epoch 9/10
6/6 

Found 120 images belonging to 3 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 856ms/step - accuracy: 1.0000 - loss: 0.0046
Test Accuracy with Transfer Learning: 100.00%
